In [ ]:
import torchvision
import torch
import pandas as pd
import torch.fx as fx

from model_static_graph import extract_graph, draw_graph
from pytorch_tracing import py_tracing_forward, py_tracing_backward


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 1000)

model = torchvision.models.resnet18().cuda()
input_data = torch.randn(5, 3, 224, 224).cuda()
model_name='resnet18'

static_graph,name_module,adj=extract_graph(model)
draw_graph(adj,name_module, model_name=model_name)
#draw_graph(adj,name_module, model_name=model_name,t=1)

py_tracing_forward(model, input_data, model_name=model_name)
py_tracing_backward(model, input_data, model_name=model_name)


/home/k8smaster/miniconda3/envs/k8s/lib/python3.10/site-packages/torch/profiler/profiler.py:445: UserWarning: Profiler won't be using warmup, this can skew profiler results
  warn("Profiler won't be using warmup, this can skew profiler results")


正在处理文件 resnet18_forward.json...
文件 resnet18_forward.json 已成功转化为 log_csv/resnet18_forward 目录下的csv
正在处理文件 resnet18.json...
文件 resnet18.json 已成功转化为 log_csv/resnet18 目录下的csv
正在处理文件 VGG16.json...
文件 VGG16.json 已成功转化为 log_csv/VGG16 目录下的csv
所有文件转换完成。
Profiling completed. Trace log saved to './log/resnet18_forward.json'
CSV files saved to 'log_csv'
正在处理文件 resnet18_backward.json...
文件 resnet18_backward.json 已成功转化为 log_csv/resnet18_backward 目录下的csv
正在处理文件 resnet18_forward.json...
文件 resnet18_forward.json 已成功转化为 log_csv/resnet18_forward 目录下的csv
正在处理文件 resnet18.json...
文件 resnet18.json 已成功转化为 log_csv/resnet18 目录下的csv
正在处理文件 VGG16.json...
文件 VGG16.json 已成功转化为 log_csv/VGG16 目录下的csv
所有文件转换完成。
Profiling completed. Trace log saved to './log/resnet18_backward.json'
CSV files saved to 'log_csv'


In [8]:
import pandas as pd

cpu=pd.read_csv('log_csv/resnet18_forward/cpu_op.csv')
cuda=pd.read_csv('log_csv/resnet18_forward/cuda_runtime.csv')
kernel=pd.read_csv('log_csv/resnet18_forward/kernel.csv')
cuda=cuda[cuda['name'].str.contains('cudaLaunchKernel')].reset_index(drop=True)

kernel_list=set(kernel['name'].to_list())
#提出cuda里的correlation和External id 两列
cuda=cuda[['correlation','External id']]
op_kernel={}
for i in range(len(cuda)):
    corr=cuda.loc[i,'correlation']
    ex_id=cuda.loc[i,'External id']
    kernel_name=kernel[kernel['correlation']==corr]['name'].values[0]
    op_name=cpu[cpu['External id']==ex_id]['name'].values[0]
    if op_name not in op_kernel:
        op_kernel[op_name]=set()
    op_kernel[op_name].add(kernel_name)

len(kernel)

133

In [6]:
total=0
for key in op_kernel:
    sum=0
    for kernel_name in op_kernel[key]:
        sum+=len(kernel[kernel['name']==kernel_name])
    print(key,len(cpu[cpu['name']==key]),"kernel_sum",sum)
    for kernel_name in op_kernel[key]:
        print("    ",kernel_name,len(kernel[kernel['name']==kernel_name]))
    total+=sum

print("total kernel",total)

aten::cudnn_convolution 20 kernel_sum 65
     void cutlass__5x_cudnn::Kernel<cutlass_tensorop_s1688fprop_optimized_tf32_128x64_16x6_nhwc_align4>(cutlass_tensorop_s1688fprop_optimized_tf32_128x64_16x6_nhwc_align4::Params) 1
     _5x_cudnn_ampere_scudnn_128x64_relu_interior_nn_v1 1
     void implicit_convolve_sgemm<float, float, 1024, 5, 5, 3, 3, 3, 1, false, false, true>(int, int, int, float const*, int, float*, float const*, kernel_conv_params, unsigned long long, int, float, float, int, float const*, float const*, bool, bool, int, int) 1
     void cudnn::engines_precompiled::nchwToNhwcKernel<float, float, float, false, true, (cudnnKernelDataType_t)2>(cudnn::engines_precompiled::nchw2nhwc_params_t<float>, float const*, float*) 34
     sm86_xmma_fprop_implicit_gemm_tf32f32_tf32f32_f32_nhwckrsc_nchw_tilesize128x64x32_stage4_warpsize2x2x1_g1_tensor16x8x8_alignc4_execute_kernel__5x_cudnn 4
     sm86_xmma_fprop_implicit_gemm_indexed_tf32f32_tf32f32_f32_nhwckrsc_nchw_tilesize64x32x64_stage4_

In [5]:
#比较op_kernel里每个key的set是否有交集
for key1 in op_kernel:
    for key2 in op_kernel:
        if key1!=key2:
            if len(op_kernel[key1]&op_kernel[key2])>0:
                print(key1,key2,op_kernel[key1]&op_kernel[key2])

In [ ]:

# tracer = fx.Tracer()
# graph = tracer.trace(model)

# # 创建 GraphModule
# graph_module = fx.GraphModule(model, graph)
# #graph_module
# #输出fx.里的方法
# dir(fx)

In [ ]:
# traced_model = fx.symbolic_trace(model)
# graph_module.print_readable()